In [1]:
#Домашнее задание Титанк

In [84]:
import numpy as np
import pandas as pd

In [85]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [86]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('gender_submission.csv')

In [87]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [88]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [89]:
y_train = train.Survived
train = train.drop(['PassengerId','Ticket','Fare','Cabin','Embarked','Survived'],axis=1)
test = test.drop(['PassengerId','Ticket','Fare','Cabin','Embarked'],axis=1)
train['is_test'] =  False
test['is_test'] = True
X_all = pd.concat([train,test],axis=0)
X_all.index = range(len(X_all))
X_all.head()

,Pclass,Name,Sex,Age,SibSp,Parch,is_test
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,False
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,False
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,False
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,False
4,3,"Allen, Mr. William Henry",male,35.0,0,0,False


Тут из имени выделяем обращение и из параметров "Parch" и "SibSp" преполагаем делаем размер семьи.
Работу с этим параметрами посмотерл на Каггле. Как человек имеющий небольшое отошение к морю знаю, что женщины  дети первыми в шлюпки и семьи стараются не разлучать.

In [90]:
pre = X_all[['Name']]
title = pre['Name'].str.split(',').str[1].str.split('.').str[0]
X_all['Title'] = title
X_all['Group_num'] = X_all.Parch + X_all.SibSp + 1
del pre
del title
X_all.head()

,Pclass,Name,Sex,Age,SibSp,Parch,is_test,Title,Group_num
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,False,Mr,2
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,False,Mrs,2
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,False,Miss,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,False,Mrs,2
4,3,"Allen, Mr. William Henry",male,35.0,0,0,False,Mr,1


In [91]:
X_all = X_all.drop(['SibSp', 'Name','Parch'],axis=1)
X_all.head()

,Pclass,Sex,Age,is_test,Title,Group_num
0,3,male,22.0,False,Mr,2
1,1,female,38.0,False,Mrs,2
2,3,female,26.0,False,Miss,1
3,1,female,35.0,False,Mrs,2
4,3,male,35.0,False,Mr,1


In [92]:
X_all['Age'].fillna((X_all['Age'].median()), inplace=True)
X_all.isnull().sum()

Pclass       0
Sex          0
Age          0
is_test      0
Title        0
Group_num    0
dtype: int64

In [93]:
X_all = pd.get_dummies(X_all, columns=['Sex','Title'])
X_all.head()

,Pclass,Age,is_test,Group_num,Sex_female,Sex_male,Title_ Capt,Title_ Col,Title_ Don,Title_ Dona,...,Title_ Master,Title_ Miss,Title_ Mlle,Title_ Mme,Title_ Mr,Title_ Mrs,Title_ Ms,Title_ Rev,Title_ Sir,Title_ the Countess
0,3,22.0,False,2,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,38.0,False,2,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,26.0,False,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,35.0,False,2,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3,35.0,False,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [94]:
X_train = X_all[~X_all.is_test].drop(['is_test'],axis=1)
X_train.head()

,Pclass,Age,Group_num,Sex_female,Sex_male,Title_ Capt,Title_ Col,Title_ Don,Title_ Dona,Title_ Dr,...,Title_ Master,Title_ Miss,Title_ Mlle,Title_ Mme,Title_ Mr,Title_ Mrs,Title_ Ms,Title_ Rev,Title_ Sir,Title_ the Countess
0,3,22.0,2,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,38.0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,26.0,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,35.0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3,35.0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [95]:
X_test = X_all[X_all.is_test].drop(['is_test'],axis=1)
X_test.head()

,Pclass,Age,Group_num,Sex_female,Sex_male,Title_ Capt,Title_ Col,Title_ Don,Title_ Dona,Title_ Dr,...,Title_ Master,Title_ Miss,Title_ Mlle,Title_ Mme,Title_ Mr,Title_ Mrs,Title_ Ms,Title_ Rev,Title_ Sir,Title_ the Countess
891,3,34.5,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
892,3,47.0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
893,2,62.0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
894,3,27.0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
895,3,22.0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [96]:
from sklearn.tree import DecisionTreeClassifier

In [97]:
from sklearn.preprocessing import LabelEncoder

In [98]:
#le = LabelEncoder()
#y = train.Survived
#le_sex = LabelEncoder()
#X_train.Sex = le_sex.fit_transform(X_train.Sex)
#X_test.Sex = le_sex.fit_transform(X_test.Sex)
#le_title = LabelEncoder()
#X_train.Title = le_title.fit_transform(X_train.Title)
#X_test.Title = le_title.fit_transform(X_test.Title)

In [99]:
X_train.head()

,Pclass,Age,Group_num,Sex_female,Sex_male,Title_ Capt,Title_ Col,Title_ Don,Title_ Dona,Title_ Dr,...,Title_ Master,Title_ Miss,Title_ Mlle,Title_ Mme,Title_ Mr,Title_ Mrs,Title_ Ms,Title_ Rev,Title_ Sir,Title_ the Countess
0,3,22.0,2,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,38.0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,26.0,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,35.0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3,35.0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [100]:
from sklearn.model_selection import GridSearchCV

In [140]:
depths = np.arange(1,10)
grid = {'max_depth': depths}
gridsearch = GridSearchCV(DecisionTreeClassifier(), grid, scoring='neg_log_loss', cv=5,return_train_score=True)

In [141]:
%%time
gridsearch.fit(X_train, y)

Wall time: 191 ms


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [142]:
print('Best Params:')
print(gridsearch.best_params_)
print('Best CV Score:')
print(gridsearch.best_score_)
print('Best CV Estimator:')
print(gridsearch.best_estimator_)
print('Mean train score:')
print(gridsearch.cv_results_['mean_train_score'])

Best Params:
{'max_depth': 2}
Best CV Score:
-0.47523105751976874
Best CV Estimator:
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Mean train score:
[-0.5093636  -0.44665259 -0.40335812 -0.37042936 -0.34076449 -0.31298342
 -0.28055217 -0.25495864 -0.22885282]


In [120]:
clf_final = gridsearch.best_estimator_

In [121]:
clf_final.fit(X_train, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [145]:
y_test = clf_final.predict(X_test)
list(y_test).count(1)

164

In [143]:
clf_final.feature_importances_

array([0.05796508, 0.        , 0.17859346, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.76344145, 0.        , 0.        ,
       0.        , 0.        , 0.        ])

In [144]:
X_test.columns

Index(['Pclass', 'Age', 'Group_num', 'Sex_female', 'Sex_male', 'Title_ Capt',
       'Title_ Col', 'Title_ Don', 'Title_ Dona', 'Title_ Dr',
       'Title_ Jonkheer', 'Title_ Lady', 'Title_ Major', 'Title_ Master',
       'Title_ Miss', 'Title_ Mlle', 'Title_ Mme', 'Title_ Mr', 'Title_ Mrs',
       'Title_ Ms', 'Title_ Rev', 'Title_ Sir', 'Title_ the Countess'],
      dtype='object')

In [124]:
submit = pd.DataFrame(y_test, columns=sample_submission.columns[1:])
submit['PassengerId'] = sample_submission.PassengerId
submit = submit[[submit.columns[-1]]+list(submit.columns[:-1])]
submit.to_csv('submit.csv', index=False)

In [125]:
from sklearn.tree import export_graphviz

def get_tree_dot_view(clf, feature_names=None, class_names=None):
    print(export_graphviz(clf, out_file=None, filled=True, feature_names=feature_names, class_names=class_names))

In [137]:
print(export_graphviz(clf_final, out_file=None, filled=True, feature_names=list(X_test.columns)))

digraph Tree {
node [shape=box, style="filled", color="black"] ;
0 [label="Title_ Mr <= 0.5\ngini = 0.473\nsamples = 891\nvalue = [549, 342]", fillcolor="#e5813960"] ;
1 [label="Group_num <= 4.5\ngini = 0.422\nsamples = 374\nvalue = [113, 261]", fillcolor="#399de591"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="gini = 0.346\nsamples = 323\nvalue = [72, 251]", fillcolor="#399de5b6"] ;
1 -> 2 ;
3 [label="gini = 0.315\nsamples = 51\nvalue = [41, 10]", fillcolor="#e58139c1"] ;
1 -> 3 ;
4 [label="Pclass <= 1.5\ngini = 0.264\nsamples = 517\nvalue = [436, 81]", fillcolor="#e58139d0"] ;
0 -> 4 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
5 [label="gini = 0.452\nsamples = 107\nvalue = [70, 37]", fillcolor="#e5813978"] ;
4 -> 5 ;
6 [label="gini = 0.192\nsamples = 410\nvalue = [366, 44]", fillcolor="#e58139e0"] ;
4 -> 6 ;
}


![alt text](Webgraphviz_new.jpg "Graf")